# Use Apache Sedona in CASD

In this notebook, we will learn
- What is apache sedona
- how to create a sedona session(via pyspark) in CASD
- how to read geospatial data
- how to do geospatial calculation
- how to write result in geoparquet

## 1. What is apache sedona?

[Apache Sedona](https://sedona.apache.org/latest/) is a cluster computing system for processing `large-scale spatial data`. In this tutorial, we will learn
how to use sedona in a `pyspark environment`. It supports other `cluster computing systems`, such as `Apache Flink, and Snowflake`. But we will not cover them in this tutorial.

### 1.1 Sedona Core (Scala)

Sedona adds support for spatial data types and operations to Spark.

It includes:

- Spatial RDDs
- Geometry types (Point, Polygon, etc.)
- Spatial indexes (QuadTree, RTree)
- Spatial operations (joins, predicates)

### 1.2 Sedona workflow

Sedona functions you call in Python (e.g., `ST_Point`, `ST_Within`) are just Python wrappers that translate calls to JVM methods.

The workflow is sedona->python->py4j->spark->jvm

## 2. Create a sedona session in CASD

To be able to create a sedona session in CASD. You need to
1. Check if you have the required jar files in your `bulle`
2. Create a python virtual environment and install the python apache-sedona dependencies
3. Create a sedona session(via pyspark) in python


### 2.1 Check the sedona required jar files
As I explained before, sedona is written in `scala`, and released as `.jar` files.

CASD provides multiple versions of sedona. But by default, CASD only offers the jar files of the `latest version of sedona`. If you want to use a specific version of sedona, you need to contact `service@casd.eu`

The location of the sedona jar file is: **\sedona-35-212-172**. Inside this folder, you can find two jar files:
- `sedona-spark-shaded-3.5_2.12-1.7.2.jar`: this sedona jar is built for `spark 3.5.*` compile with `scala 2.12`. The `sedona version is 1.7.2` (latest on 11/08/2025)
- `geotools-wrapper-1.7.2-28.5.jar`: this geotools jar is built for `sedona version 1.7.2`, the `geotools version is 28.5`

You can find the origin of the jar files in the below urls:
- sedona-jars: https://repo.maven.apache.org/maven2/org/apache/sedona/
- geotools-wrapper-jars: https://repo.maven.apache.org/maven2/org/datasyslab/geotools-wrapper/

### 2.2 Install apache-sedona python dependency

Here, we suppose you have completed the tutorial `01.Create_python_env_in_CASD` and `02.Use_pyspark_in_CASD`.
In another word, you already installed spark, and you have a `python virtual environment` which contains `pyspark`

```shell
# To install pyspark along with Sedona Python in one go, use the spark extra
pip install apache-sedona[spark]

# you need to check the version of apacke-sedona, because the jar version must be compatible 
pip show apache-sedona
```

### 2.3 Create a sedona[spark] context

As I explained before, sedona spark context is built on top of spark. So we need to create a spark session first in order to create a sedona context.

In [4]:
from sedona.spark import *
from pathlib import Path
from pyspark.sql import SparkSession, DataFrame

# import sedona constructor function, often used to build geometry type column from native data type 
from sedona.sql import st_constructors as stc

# import sedona simple function, often used to do geometry calculation such as ST_Distance, 
from sedona.sql import st_functions as stf

# import sedona predicates function, often used to determine relation between two geometry column such as ST_Contains
from sedona.sql import st_predicates as stp
# import sedona aggregates function,
from sedona.sql import st_aggregates

In [5]:
# build a sedona session offline
project_root_dir = Path.cwd().parent

print(project_root_dir.as_posix())

C:/Users/PLIU/Documents/git/Seminar_PySpark_Sedona_GeoParquet


In [6]:
# here we choose sedona 1.7.2 for spark 3.5.* build with scala 2.12
jar_folder = Path(f"{project_root_dir}/jars/sedona-35-212-172")
jar_list = [str(jar) for jar in jar_folder.iterdir() if jar.is_file()]
jar_path = ",".join(jar_list)

# build a sedona session (sedona = 1.7.2) offline
spark = SparkSession.builder \
    .appName("sedona_tutorial") \
    .master("local[*]") \
    .config("spark.jars", jar_path) \
    .getOrCreate()


In [7]:
# create a sedona context
sedona = SedonaContext.create(spark)

In [10]:
# get the spark context
sc = spark.sparkContext
# use utf as default encoding
sc.setSystemProperty("sedona.global.charset", "utf8")

## 3. Read geospatial data with sedona

In this section, we will read two shape files:
- airports_shape: all international airports in the world. The geo column is a point
- counties_shape: all countries in the world. The geo column is a multi-polygon

In [12]:
data_dir = f"{project_root_dir}/data"
airports_file_path = f"{data_dir}/airports_shape"
countries_file_path = f"{data_dir}/countries_shape"

In [16]:
# read countries shape file
countries_df = sedona.read.format("shapefile").load(countries_file_path)
countries_df.printSchema()

root
 |-- geometry: geometry (nullable = true)
 |-- featurecla: string (nullable = true)
 |-- scalerank: long (nullable = true)
 |-- LABELRANK: long (nullable = true)
 |-- SOVEREIGNT: string (nullable = true)
 |-- SOV_A3: string (nullable = true)
 |-- ADM0_DIF: long (nullable = true)
 |-- LEVEL: long (nullable = true)
 |-- TYPE: string (nullable = true)
 |-- ADMIN: string (nullable = true)
 |-- ADM0_A3: string (nullable = true)
 |-- GEOU_DIF: long (nullable = true)
 |-- GEOUNIT: string (nullable = true)
 |-- GU_A3: string (nullable = true)
 |-- SU_DIF: long (nullable = true)
 |-- SUBUNIT: string (nullable = true)
 |-- SU_A3: string (nullable = true)
 |-- BRK_DIFF: long (nullable = true)
 |-- NAME: string (nullable = true)
 |-- NAME_LONG: string (nullable = true)
 |-- BRK_A3: string (nullable = true)
 |-- BRK_NAME: string (nullable = true)
 |-- BRK_GROUP: string (nullable = true)
 |-- ABBREV: string (nullable = true)
 |-- POSTAL: string (nullable = true)
 |-- FORMAL_EN: string (nullable

In [19]:
countries_df.show(1, vertical=True)

-RECORD 0--------------------------
 geometry   | MULTIPOLYGON (((3... 
 featurecla | Admin-0 country      
 scalerank  | 1                    
 LABELRANK  | 3                    
 SOVEREIGNT | Zimbabwe             
 SOV_A3     | ZWE                  
 ADM0_DIF   | 0                    
 LEVEL      | 2                    
 TYPE       | Sovereign country    
 ADMIN      | Zimbabwe             
 ADM0_A3    | ZWE                  
 GEOU_DIF   | 0                    
 GEOUNIT    | Zimbabwe             
 GU_A3      | ZWE                  
 SU_DIF     | 0                    
 SUBUNIT    | Zimbabwe             
 SU_A3      | ZWE                  
 BRK_DIFF   | 0                    
 NAME       | Zimbabwe             
 NAME_LONG  | Zimbabwe             
 BRK_A3     | ZWE                  
 BRK_NAME   | Zimbabwe             
 BRK_GROUP  |                      
 ABBREV     | Zimb.                
 POSTAL     | ZW                   
 FORMAL_EN  | Republic of Zimbabwe 
 FORMAL_FR  |               

In [17]:
# read airports shape file
airports_df = sedona.read.format("shapefile").load(airports_file_path)
airports_df.printSchema()

root
 |-- geometry: geometry (nullable = true)
 |-- scalerank: long (nullable = true)
 |-- featurecla: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- abbrev: string (nullable = true)
 |-- location: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- wikipedia: string (nullable = true)
 |-- natlscale: decimal(8,3) (nullable = true)



In [18]:
airports_df.show(1, vertical=True)

-RECORD 0--------------------------
 geometry   | POINT (113.935016... 
 scalerank  | 2                    
 featurecla | Airport              
 type       | major                
 name       | Hong Kong Int'l      
 abbrev     | HKG                  
 location   | terminal             
 gps_code   | VHHH                 
 iata_code  | HKG                  
 wikipedia  | http://en.wikiped... 
 natlscale  | 150.000              
only showing top 1 row



## 4. Use sedona geo function

In this example, we join the country data frame and airport data frame by using the condition **ST_Contains(c.geometry, a.geometry)**. It means if the `airport (point)` in the `country (polygon)`, then we link the two rows.

In [20]:
from pyspark.sql.functions import col

# create a new dataframe to host the result of the join
countries_airport_df = (
    countries_df.alias("c")
    .join(
        airports_df.alias("a"),
        ST_Contains(col("c.geometry"), col("a.geometry"))
    )
    .select(
        col("c.geometry").alias("country_location"),
        col("c.NAME_EN").alias("country_name"),
        col("a.geometry").alias("airport_location"),
        col("a.name").alias("airport_name")
    )
)

countries_airport_df.show(5, truncate=False)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [21]:
countries_airport_df.printSchema()

root
 |-- country_location: geometry (nullable = true)
 |-- country_name: string (nullable = true)
 |-- airport_location: geometry (nullable = true)
 |-- airport_name: string (nullable = true)



In [16]:
# create a new view to store the result of the geo join
countries_airport_df.createOrReplaceTempView("country_airport")

In [27]:
from pyspark.sql.functions import count, desc

airports_count = (countries_airport_df
                  .groupBy("country_name", "country_location")
                  .agg(count("*").alias("airport_count"))
                  .sort(desc("airport_count")))

airports_count.show(5)

+--------------------+--------------------+-------------+
|        country_name|    country_location|airport_count|
+--------------------+--------------------+-------------+
|United States of ...|MULTIPOLYGON (((-...|           35|
|              Canada|MULTIPOLYGON (((-...|           15|
|              Mexico|MULTIPOLYGON (((-...|           12|
|              Brazil|MULTIPOLYGON (((-...|           12|
|People's Republic...|MULTIPOLYGON (((1...|            7|
+--------------------+--------------------+-------------+
only showing top 5 rows



In [23]:
france_airports = (
    countries_airport_df
    .filter(col("country_name").like("%France%"))
    .select(
        col("country_name"),
        col("airport_name"),
        col("airport_location")
    )
)
france_airports.show(5, truncate=False)

+------------+-----------------------+--------------------------------------------+
|country_name|airport_name           |airport_location                            |
+------------+-----------------------+--------------------------------------------+
|France      |Paris Orly             |POINT (2.367379127837731 48.73130304580517) |
|France      |Charles de Gaulle Int'l|POINT (2.5418677673945727 49.01442009693855)|
+------------+-----------------------+--------------------------------------------+



In [24]:
france_airports.printSchema()

root
 |-- country_name: string (nullable = true)
 |-- airport_name: string (nullable = true)
 |-- airport_location: geometry (nullable = true)



## 5 Write result in a geoparquet file

In [28]:
out_path = f"{data_dir}/tmp/france_airports"
france_airports.write.mode("overwrite").format("geoparquet").save(out_path)

In [29]:
tmp_df = sedona.read.format("geoparquet").load(out_path)
tmp_df.show(5, truncate=False)

+------------+-----------------------+--------------------------------------------+
|country_name|airport_name           |airport_location                            |
+------------+-----------------------+--------------------------------------------+
|France      |Paris Orly             |POINT (2.367379127837731 48.73130304580517) |
|France      |Charles de Gaulle Int'l|POINT (2.5418677673945727 49.01442009693855)|
+------------+-----------------------+--------------------------------------------+



In [30]:
tmp_df.printSchema()

root
 |-- country_name: string (nullable = true)
 |-- airport_name: string (nullable = true)
 |-- airport_location: geometry (nullable = true)

